In [ ]:
!pip install gwpy
%tensorflow_version 1.x

In [3]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
import sys
from tqdm import tqdm

from gwpy.timeseries import TimeSeries
import time
from gwpy.time import tconvert
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# RBM Implementation


In [5]:
# Copyright (c) 2016 Michal Lukac, Egor Malykh
# Todo its an MIT license so we can use it but check whats about the changes and where to credit

class RBM:
    def __init__(self,
                 n_visible,
                 n_hidden,
                 learning_rate=0.01,
                 momentum=0.95,
                 xavier_const=1.0,
                 err_function='cosine',
                 use_tqdm=False):

        self.sample_visible = False
        self.sigma = 1
        if not 0.0 <= momentum <= 1.0:
            raise ValueError('momentum should be in range [0, 1]')

        if err_function not in {'mse', 'cosine'}:
            raise ValueError('err_function should be either \'mse\' or \'cosine\'')

        self._use_tqdm = use_tqdm
        self._tqdm = None

        if use_tqdm or tqdm is not None:
            self._tqdm = tqdm
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.learning_rate = learning_rate
        self.momentum = momentum

        self.x = tf.placeholder(tf.float32, [None, self.n_visible])
        self.y = tf.placeholder(tf.float32, [None, self.n_hidden])

        self.w = tf.Variable(tf_xavier_init(n_visible, n_hidden), dtype=tf.float32)
        self.visible_bias = tf.Variable(tf.zeros([self.n_visible]), dtype=tf.float32)
        self.hidden_bias = tf.Variable(tf.zeros([self.n_hidden]), dtype=tf.float32)

        self.delta_w = tf.Variable(tf.zeros([self.n_visible, self.n_hidden]), dtype=tf.float32)
        self.delta_visible_bias = tf.Variable(tf.zeros([self.n_visible]), dtype=tf.float32)
        self.delta_hidden_bias = tf.Variable(tf.zeros([self.n_hidden]), dtype=tf.float32)

        self.update_weights = None
        self.update_deltas = None
        self.compute_hidden = None
        self.compute_visible = None
        self.compute_visible_from_hidden = None

        self._initialize_vars()

        assert self.update_weights is not None
        assert self.update_deltas is not None
        assert self.compute_hidden is not None
        assert self.compute_visible is not None
        assert self.compute_visible_from_hidden is not None

        # Todo change error function
        if err_function == 'cosine':
            x1_norm = tf.nn.l2_normalize(self.x, 1)
            x2_norm = tf.nn.l2_normalize(self.compute_visible, 1)
            cos_val = tf.reduce_mean(tf.reduce_sum(tf.math.multiply(x1_norm, x2_norm), 1))
            self.compute_err = tf.acos(cos_val) / tf.constant(np.pi)
        else:
            self.compute_err = tf.reduce_mean(tf.square(self.x - self.compute_visible))

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)

    def _initialize_vars(self):
        hidden_p = tf.nn.sigmoid(tf.matmul(self.x, self.w) + self.hidden_bias)
        visible_recon_p = tf.matmul(sample_bernoulli(hidden_p), tf.transpose(self.w)) + self.visible_bias

        if self.sample_visible:
            visible_recon_p = sample_gaussian(visible_recon_p, self.sigma)

        hidden_recon_p = tf.nn.sigmoid(tf.matmul(visible_recon_p, self.w) + self.hidden_bias)

        positive_grad = tf.matmul(tf.transpose(self.x), hidden_p)
        negative_grad = tf.matmul(tf.transpose(visible_recon_p), hidden_recon_p)

        def f(x_old, x_new):
            return self.momentum * x_old + \
                   self.learning_rate * x_new * (1 - self.momentum) / tf.to_float(tf.shape(x_new)[0])

        delta_w_new = f(self.delta_w, positive_grad - negative_grad)
        delta_visible_bias_new = f(self.delta_visible_bias, tf.reduce_mean(self.x - visible_recon_p, 0))
        delta_hidden_bias_new = f(self.delta_hidden_bias, tf.reduce_mean(hidden_p - hidden_recon_p, 0))

        update_delta_w = self.delta_w.assign(delta_w_new)
        update_delta_visible_bias = self.delta_visible_bias.assign(delta_visible_bias_new)
        update_delta_hidden_bias = self.delta_hidden_bias.assign(delta_hidden_bias_new)

        update_w = self.w.assign(self.w + delta_w_new)
        update_visible_bias = self.visible_bias.assign(self.visible_bias + delta_visible_bias_new)
        update_hidden_bias = self.hidden_bias.assign(self.hidden_bias + delta_hidden_bias_new)

        self.update_deltas = [update_delta_w, update_delta_visible_bias, update_delta_hidden_bias]
        self.update_weights = [update_w, update_visible_bias, update_hidden_bias]

        self.compute_hidden = tf.nn.sigmoid(tf.matmul(self.x, self.w) + self.hidden_bias)
        self.compute_visible = tf.matmul(self.compute_hidden, tf.transpose(self.w)) + self.visible_bias
        self.compute_visible_from_hidden = tf.matmul(self.y, tf.transpose(self.w)) + self.visible_bias

    def get_err(self, batch_x):
        return self.sess.run(self.compute_err, feed_dict={self.x: batch_x})

    def get_free_energy(self, sample):
        wx_b = tf.tensordot(sample, self.w) + self.hidden_bias
        vbias_term = tf.tensordot(sample, self.visible_bias)
        hidden_term = tf.reduce_sum(tf.math.log(1 + tf.math.exp(wx_b)), axis=1)
        return -hidden_term - vbias_term

    def transform(self, batch_x):
        return self.sess.run(self.compute_hidden, feed_dict={self.x: batch_x})

    def transform_inv(self, batch_y):
        return self.sess.run(self.compute_visible_from_hidden, feed_dict={self.y: batch_y})

    def reconstruct(self, batch_x):
        return self.sess.run(self.compute_visible, feed_dict={self.x: batch_x})

    def partial_fit(self, batch_x):
        self.sess.run(self.update_weights + self.update_deltas, feed_dict={self.x: batch_x})

    def fit(self,
            data_x,
            n_epoches=10,
            batch_size=10,
            shuffle=False,
            verbose=True):
        assert n_epoches > 0

        n_data = data_x.shape[0]

        if batch_size > 0:
            n_batches = n_data // batch_size + (0 if n_data % batch_size == 0 else 1)
        else:
            n_batches = 1

        if shuffle:
            data_x_cpy = data_x.copy()
            inds = np.arange(n_data)
        else:
            data_x_cpy = data_x

        errs = []

        for e in range(n_epoches):
            if verbose and not self._use_tqdm:
                print('Epoch: {:d}'.format(e))

            epoch_errs = np.zeros((n_batches,))
            epoch_errs_ptr = 0

            if shuffle:
                np.random.shuffle(inds)
                data_x_cpy = data_x_cpy[inds]

            r_batches = range(n_batches)

            if verbose and self._use_tqdm:
                r_batches = self._tqdm(r_batches, desc='Epoch: {:d}'.format(e), ascii=True, file=sys.stdout)

            for b in r_batches:
                batch_x = data_x_cpy[b * batch_size:(b + 1) * batch_size]
                self.partial_fit(batch_x)
                batch_err = self.get_err(batch_x)
                epoch_errs[epoch_errs_ptr] = batch_err
                epoch_errs_ptr += 1

            if verbose:
                err_mean = epoch_errs.mean()
                if self._use_tqdm:
                    self._tqdm.write('Train error: {:.4f}'.format(err_mean))
                    self._tqdm.write('')
                else:
                    print('Train error: {:.4f}'.format(err_mean))
                    print('')
                sys.stdout.flush()

            errs = np.hstack([errs, epoch_errs])

        return errs

    def get_weights(self):
        return self.sess.run(self.w), \
               self.sess.run(self.visible_bias), \
               self.sess.run(self.hidden_bias)

    def save_weights(self, filename, name):
        saver = tf.train.Saver({name + '_w': self.w,
                                name + '_v': self.visible_bias,
                                name + '_h': self.hidden_bias})
        return saver.save(self.sess, filename)

    def set_weights(self, w, visible_bias, hidden_bias):
        self.sess.run(self.w.assign(w))
        self.sess.run(self.visible_bias.assign(visible_bias))
        self.sess.run(self.hidden_bias.assign(hidden_bias))

    def load_weights(self, filename, name):
        saver = tf.train.Saver({name + '_w': self.w,
                                name + '_v': self.visible_bias,
                                name + '_h': self.hidden_bias})
        saver.restore(self.sess, filename)


def tf_xavier_init(fan_in, fan_out, const=1.0, dtype=np.float32):
    k = const * np.sqrt(6.0 / (2 + int(fan_out)))
    return tf.random_uniform(np.array([fan_in, fan_out]), minval=-k, maxval=k, dtype=dtype)


def sample_bernoulli(probs):
    return tf.nn.relu(tf.sign(probs - tf.random_uniform(tf.shape(probs))))


def sample_gaussian(x, sigma):
    return x + tf.random_normal(tf.shape(x), mean=0.0, stddev=sigma, dtype=tf.float32)


# Prepare Data

In [ ]:
#change the top two parameters (play around with different frequencies, but higher might result in crashed due to memory usage)
sample_frequency = 1024
n_samples = 1000
start = 1187529256

def pretty(t):
  return tconvert(t).strftime("%a, %d %b %Y %H:%M:%S %Z")

def fetch(i=0, start=1126259446, delta=32):
  halfdelta=delta/2
  s,e=start+(delta*i)-halfdelta, start+(delta*(i+1))-halfdelta
  print("Fetching data for {0} to {1}".format(pretty(s), pretty(e)))
  hdata = TimeSeries.fetch_open_data('H1', s, e, cache=True)
  ldata = TimeSeries.fetch_open_data('L1', s, e, cache=True)
  # print("{0} points for H, {1} points for L".format(len(hdata), len(ldata)))
  return (hdata, ldata)

def filter_gwe(data:TimeSeries):
  from gwpy.signal import filter_design
  bp = filter_design.bandpass(50, 250, data.sample_rate)
  notches = [filter_design.notch(line, data.sample_rate) for line in (60, 120, 180)]
  zpk = filter_design.concatenate_zpks(bp, *notches)
  filt = data.filter(zpk, filtfilt=True)
  data = data.crop(*data.span.contract(1))
  filt = filt.crop(*filt.span.contract(1))
  resample = filt.resample(sample_frequency)
  return resample

def scale(data):
  avg = np.average(data)
  sigma = np.std(data)
  return (data - avg) / sigma

print("Sampling", n_samples, "datapoints at",sample_frequency,"Hz")
data_h, data_l = fetch(start=start, delta=8)
test_sample = filter_gwe(data_h)
data_points = len(test_sample)
x_train = np.zeros((n_samples,data_points*2))
for i in range(1,n_samples):
  e = start + (8 * i)
  data_h, data_l = fetch(start=e, delta=8)
  filt_h = np.asarray(filter_gwe(data_h))
  filt_h = scale(filt_h)
  filt_l = np.asarray(filter_gwe(data_l))
  filt_l = scale(filt_l)
  x_train[i,0:len(filt_h)] = filt_h
  x_train[i,len(filt_h):] = filt_l
  print(i)
#   # plt.subplot(3,1,1)
#   # plt.plot(data)
#   # plt.subplot(3,1,2)
#   plt.plot(filt)
#   # plt.subplot(3,1,3)
#   # (data.spectrogram(3, fftlength=3, overlap=2) ** (1/2.)).plot()
#   plt.show()

# Train RBM

In [ ]:
#first parameter is the number of visible units (restriced by data format) & the number of hidden units is the second parameter
rbm = RBM((data_points*2), (data_points*3), learning_rate=0.01, momentum=0.95, xavier_const=1.0, use_tqdm=True)
rbm.fit(x_train, n_epoches=10, batch_size=10)
# Check saving works befor training for a long time!!!
rbm.save_weights(filename='ML/rbm-weights', name='rbm-weights')